In [1]:
!pip install tner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.2 MB 8.9 MB/s 
     |████████████████████████████████| 730 kB 47.7 MB/s 
     |████████████████████████████████| 5.8 MB 63.0 MB/s 
     |████████████████████████████████| 1.3 MB 57.9 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 451 kB 43.7 MB/s 
     |████████████████████████████████| 593 kB 48.6 MB/s 
     |████████████████████████████████| 182 kB 62.2 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
     |████████████████████████████████| 125 kB 90.1 MB/s 
     |████████████████████████████████| 306 kB 65.0 MB/s 
     |████████████████████████████████| 776.3 MB 1.2 kB/s 
     |████████████████████████████████| 4.5 MB 62.7 MB/s 
     |████████████████████████████████| 1.8 MB 62.3 MB/s 
     |████████████████████████████████| 880 kB 61.7 MB/s 
     |█████████████████████████████

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cross-encoder/qnli-distilroberta-base")
qa = AutoModelForSequenceClassification.from_pretrained("cross-encoder/qnli-distilroberta-base")



In [1]:
from tner import TransformersNER
aer = TransformersNER("drive/MyDrive/nlp-course/best_model", crf=True, label2id={'B-ART': 0, 'I-ART': 1, 'O': 2})

In [2]:
from pathlib import Path
import pickle

with open(Path('drive', 'MyDrive', 'nlp-course', 'mystery.pickle'), 'rb') as f:
  stories = pickle.load(f)

In [56]:
stories[0][-1]

'Sherlock Holmes wakes up Dr. Watson early one winter morning to rush to a murder scene at the Abbey Grange near Chislehurst, Kent. Sir Eustace Brackenstall has been killed, apparently by burglars. Inspector Stanley Hopkins believes that it was the infamous Randall gang who have committed several other burglaries in the neighborhood.At Abbey Grange, Lady Brackenstall tells Holmes that her marriage was not happy; Sir Eustace was a violent, abusive drunkard. She then tells that about 11 oclock, in the dining room, she encountered an elderly man coming in the French window, followed by two younger men. The older man struck her in the face, knocking her out. When she came to, she was gagged and tied to an oaken chair with the bellrope, which they had torn down. Sir Eustace came into the room and rushed at the intruders, one of whom struck and killed him with a poker. Lady Brackenstall fainted again for a minute or two. She saw the intruders drinking wine from a bottle taken from the sidebo

In [53]:
story = stories[0][-1]
story = story.split('.')

context = 10
story = ['.'.join(story[max(0,i-context):i+context]) for i in range(len(story))]

In [54]:
from tqdm import tqdm

aers = []
for window in story[0:-context]:
  aers.extend(aer.predict([window])['entity_prediction'])



100%|██████████| 1/1 [00:01<00:00,  1.68s/it]


In [55]:
filtered_aers = sum(aers, start=[])
filtered_aers = list(set(map(lambda v: ' '.join(v['entity']), filter(lambda v: sum(v['probability'])/len(v['probability']) > 0.9, filtered_aers))))

filtered_aers = list(set(filtered_aers))
filtered_aers

['',
 'Captain Crocker',
 'Holmes,',
 'the',
 'wine bottle and glasses',
 'the bellrope,',
 'killed',
 'a',
 'long corkscrew',
 'Lady Brackenstall and',
 'the cork',
 'that',
 'with',
 'about Sir Eustace:',
 'corpse',
 'the bellrope',
 'bellrope,',
 'Lady Brackenstall',
 'older',
 'Crocker',
 'him',
 'pond.',
 'pond',
 'bellrope',
 'Lady Brackenstall,',
 'warning Crocker',
 'Hopkins',
 'Holmes',
 'Lady Brackenstalls',
 'the corkscrew']

In [ ]:
import torch
input = tokenizer(['Does Lady Breckenstall interact with the chair?'], [''.join(story[:10])], padding=True, truncation=True, return_tensors="pt")

qa.eval()
with torch.no_grad():
    scores = torch.sigmoid(qa(**input).logits)
    print(scores)

tensor([[0.6856]])
